####Fine-tuning en-indic model of indicTrans:

In [ ]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 927.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 104.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all t

In [ ]:
#Making a separate directory for keeping finetuning data and model

!mkdir /content/finetuning

In [ ]:
%cd /content/finetuning

/content/finetuning


In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 697, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 697 (delta 278), reused 344 (delta 240), pack-reused 297
Receiving objects: 100% (697/697), 2.64 MiB | 9.82 MiB/s, done.
Resolving deltas: 100% (405/405), done.
/content/finetuning/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1362, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 1362 (delta 111), reused 98 (delta 93), pack-reused 1219
Receiving objects: 100% (1362/1362), 9.56 MiB | 9.60 MiB/s, done.
Resolving deltas: 100% (721/721), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 14

In [ ]:
! sudo apt install tree
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 43.0 kB of archives.
After this operation, 115 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tree amd64 1.8.0-1 [43.0 kB]
Fetched 43.0 kB in 0s (139 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 122518 files and directories currently installe

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
# downloading the en-indic model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
!unzip en2indic.zip

--2023-05-04 06:22:48--  https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
Resolving ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)... 164.52.210.97, 101.53.152.33, 101.53.152.30, ...
Connecting to ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)|164.52.210.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4811880516 (4.5G) [application/zip]
Saving to: ‘en2indic.zip’

en2indic.zip        100%[===================>]   4.48G  18.1MB/s    in 4m 39s  

2023-05-04 06:27:29 (16.4 MB/s) - ‘en2indic.zip’ saved [4811880516/4811880516]

Archive:  en2indic.zip
   creating: en-indic/
   creating: en-indic/vocab/
  inflating: en-indic/vocab/bpe_codes.32k.SRC  
  inflating: en-indic/vocab/vocab.SRC  
  inflating: en-indic/vocab/vocab.TGT  
  inflating: en-indic/vocab/bpe_codes.32k.TGT  
   creating: en-indic/final_bin/
  inflating: en-indic/final_bin/prepro

In [ ]:
%cd /content/finetuning
!wget "https://drive.google.com/uc?id=1YFzRxmOe9BVCcE0LRZvPiefrNiqVkY7Z&export=download" -O dataset.zip
!unzip dataset.zip
%cd /content/finetuning/indicTrans

/content/finetuning
--2023-05-04 06:28:45--  https://drive.google.com/uc?id=1YFzRxmOe9BVCcE0LRZvPiefrNiqVkY7Z&export=downloa
Resolving drive.google.com (drive.google.com)... 108.177.96.102, 108.177.96.138, 108.177.96.113, ...
Connecting to drive.google.com (drive.google.com)|108.177.96.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-a4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/r3cv0dvgkj5eqn30td14bc4t8d00ql1r/1683181725000/13340322011215046102/*/1YFzRxmOe9BVCcE0LRZvPiefrNiqVkY7Z?e=downloa&uuid=96f4b385-9f51-4c62-9323-7f6b9ebdcbbb [following]
--2023-05-04 06:28:47--  https://doc-08-a4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/r3cv0dvgkj5eqn30td14bc4t8d00ql1r/1683181725000/13340322011215046102/*/1YFzRxmOe9BVCcE0LRZvPiefrNiqVkY7Z?e=downloa&uuid=96f4b385-9f51-4c62-9323-7f6b9ebdcbbb
Resolving doc-08-a4-docs.googleusercontent.com (doc-08-a4-docs.googleusercontent.com)... 108.

In [ ]:
%%shell
# Creating working directory and copying data into it:
# exp_dir: Directory in which preprocessed data is stored
# download_dir: Directory in which extracted en-indic model is stored
# Copy the vocab from downloaded model to working directory

exp_dir=/content/finetuning/dataset_finetune
download_dir=/content/finetuning/en-indic
src_lang=en
tgt_lang=mr
cp -r $download_dir/vocab $exp_dir

echo "Running experiment ${exp_dir} on ${src_lang} to ${tgt_lang}"

# Preparing directory structure:
# train_processed_dir, devtest_processed_dir: For processed data
# train_norm_dir, devtest_norm_dir: For Normalization output

train_processed_dir=$exp_dir/data
devtest_processed_dir=$exp_dir/data

mkdir -p $train_processed_dir
mkdir -p $devtest_processed_dir

train_norm_dir=$exp_dir/norm/$src_lang-$tgt_lang
devtest_norm_dir=$exp_dir/norm/$src_lang-$tgt_lang

mkdir -p $train_norm_dir
mkdir -p $devtest_norm_dir


# Pre-processing of train, dev and test set

datasets=(train dev test)

for dataset in ${datasets[@]};do
		if [ $dataset == train ]; then
			in_dir=$exp_dir/$dataset/$src_lang-$tgt_lang
		else
			in_dir=$exp_dir/$dataset
		fi

		op_dir=$exp_dir/norm/$src_lang-$tgt_lang

    infname_src=$in_dir/$dataset.$src_lang
    infname_tgt=$in_dir/$dataset.$tgt_lang

		outfname_src=$op_dir/$dataset.$src_lang
    outfname_tgt=$op_dir/$dataset.$tgt_lang

    echo "Applying normalization and script conversion for $dataset $lang"

    input_size=`python scripts/preprocess_translate.py $infname_src $outfname_src $src_lang true`
    input_size=`python scripts/preprocess_translate.py $infname_tgt $outfname_tgt $tgt_lang true`

    echo "Number of sentences in $dataset $lang: $input_size"
done

#Concatenate Data
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'train'
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'dev'
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'test'

#Apply Byte Pair Encoding (BPE):
#BPE is a form of data compression algorithm in which the most common pair of consecutive bytes of data is replaced by the by the byte that has not occured in that data.
echo "Applying bpe to the new finetuning data"
bash apply_single_bpe_traindevtest_notag.sh $exp_dir

#Create Final Prepared Data Dir from which Binarizer will take input, this dir will contain data with special language tags
bin_input_data_dir=$exp_dir/final
mkdir -p $bin_input_data_dir

# Adding special language tags to indicate the source and the target language
#Hence, to translate from English to Marathi, give input as _src_en_ _tgt_mr_

echo "Adding language tags"
python scripts/add_joint_tags_translate.py $exp_dir 'train'
python scripts/add_joint_tags_translate.py $exp_dir 'dev'
python scripts/add_joint_tags_translate.py $exp_dir 'test'

#Preparing directory to store binarized output
bin_out_data_dir=$exp_dir/final_bin

rm -rf $bin_out_data_dir

#To get number of cores processing simultaneously
num_workers=`python -c "import multiprocessing; print(multiprocessing.cpu_count())"`

#Binarization of data
echo "Binarizing data"
fairseq-preprocess --source-lang SRC --target-lang TGT \
 --trainpref $bin_input_data_dir/train --validpref $bin_input_data_dir/dev --testpref $bin_input_data_dir/test \
 --destdir $bin_out_data_dir --workers $num_workers \
 --srcdict $download_dir/final_bin/dict.SRC.txt --tgtdict $download_dir/final_bin/dict.TGT.txt --thresholdtgt 5 --thresholdsrc 5

Running experiment /content/finetuning/dataset_finetune on en to mr
Applying normalization and script conversion for train 
100% 5168/5168 [00:01<00:00, 4458.91it/s]
100% 5168/5168 [00:00<00:00, 9831.83it/s]
Number of sentences in train : 5168
Applying normalization and script conversion for dev 
100% 1000/1000 [00:00<00:00, 2111.34it/s]
100% 1000/1000 [00:00<00:00, 5436.85it/s]
Number of sentences in dev : 1000
Applying normalization and script conversion for test 
100% 1000/1000 [00:00<00:00, 2231.15it/s]
100% 1000/1000 [00:00<00:00, 6240.39it/s]
Number of sentences in test : 1000

/content/finetuning/dataset_finetune/data/train.SRC
/content/finetuning/dataset_finetune/data/train.TGT
  0% 0/11 [00:00<?, ?it/s]src: en, tgt:as
src: en, tgt:bn
src: en, tgt:gu
src: en, tgt:hi
src: en, tgt:kn
src: en, tgt:ml
src: en, tgt:mr
/content/finetuning/dataset_finetune/norm/en-mr/train.en
/content/finetuning/dataset_finetune/norm/en-mr/train.mr
src: en, tgt:or
src: en, tgt:pa
src: en, tgt:ta
src: 

In [ ]:
!( fairseq-train ../dataset_finetune/final_bin \
--max-source-positions=210 \
--max-target-positions=210 \
--max-update=1000 \
--save-interval=1 \
--arch=transformer_4x \
--criterion=label_smoothed_cross_entropy \
--source-lang=SRC \
--lr-scheduler=inverse_sqrt \
--target-lang=TGT \
--label-smoothing=0.1 \
--optimizer adam \
--adam-betas "(0.9, 0.98)" \
--clip-norm 1.0 \
--warmup-init-lr 1e-07 \
--warmup-updates 4000 \
--dropout 0.2 \
--tensorboard-logdir ../dataset_finetune/tensorboard-wandb \
--save-dir ../dataset_finetune/model \
--keep-last-epochs 5 \
--patience 5 \
--skip-invalid-size-inputs-valid-test \
--user-dir model_configs \
--update-freq=2 \
--distributed-world-size 1 \
--max-tokens 256 \
--lr 3e-5 \
--restore-file ../en-indic/model/checkpoint_best.pt \
--reset-lr-scheduler \
--reset-meters \
--reset-dataloader \
--reset-optimizer)

2023-05-04 06:29:25.361415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-04 06:29:26 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-05-04 06:29:30 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': '../dataset_finetune/tensorboard-wandb', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': 'model_configs', 'empty_cache_freq'

In [ ]:
from finetuning.indicTrans.inference.engine import Model
en2indic_model_base = Model(expdir='/content/finetuning/en-indic')
en2indic_model_finetuned = Model(expdir='/content/finetuning/dataset_finetune')

In [ ]:
# import torch
# checkpoint = torch.load('/content/finetuning/dataset/model/checkpoint_best.pt')

In [ ]:
# en2indic_model_finetuned.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
f=open("/content/finetuning/dataset_finetune/test/test.en","r")
test_en_list_full=[]
for x in f:
  test_en_list_full.append(x)

test_en_list=test_en_list_full

f=open("/content/finetuning/dataset_finetune/test/test.mr","r")
test_mr_list_full=[]
for x in f:
  test_mr_list_full.append(x)

test_mr_list=test_mr_list_full

In [ ]:
BATCH_SIZE = 2
NUM_BATCHES = len(test_en_list)//BATCH_SIZE + 1
predictions_mr_finetuned=[]
predictions_mr_base=[]
for i in range(NUM_BATCHES):
  en_sents_batch = test_en_list[BATCH_SIZE*i : BATCH_SIZE*(i+1)]
  predictions_mr_base.extend(en2indic_model_base.batch_translate(en_sents_batch,'en','mr'))
  predictions_mr_finetuned.extend(en2indic_model_finetuned.batch_translate(en_sents_batch,'en','mr'))

In [ ]:
ref_mr_list=[[x] for x in test_mr_list]

In [ ]:
from sacrebleu.metrics import BLEU
bleu = BLEU()

In [ ]:
bleu.corpus_score(predictions_mr_base, ref_mr_list)

In [ ]:
bleu.corpus_score(predictions_mr_finetuned, ref_mr_list)